In [73]:
from datasets import load_dataset
import json
import openai

from model_costs import calc_cost
from helpers import extract_code

# Load the full dataset
dataset = load_dataset("codeparrot/apps", split="test")

# Load the interview JSON file
with open("introductory.json") as f:
   introductory_json = json.load(f)

# Remove leading zeros and convert to string
introductory_json = [str(int(i)) for i in introductory_json]

# Use a single filter operation instead of a loop
# This assumes 'problem_id' is a column in your dataset.
filtered_data = dataset.filter(lambda example: str(example["problem_id"]) in introductory_json)


In [74]:
sample = filtered_data[3]
sample["solutions"] = json.loads(sample["solutions"])
sample["input_output"] = json.loads(sample["input_output"])
print(sample.keys())

dict_keys(['problem_id', 'question', 'solutions', 'input_output', 'difficulty', 'url', 'starter_code'])


In [75]:
system_message = """You are an expert Python programmer. You will be given a question (problem specification) and will generate a
correct Python program that matches the specification and passes all tests. You will NOT return
anything except for the program. Put your fixed program within code delimiters, for example: ```python```. Do not call the function, just define it."""
user_message = sample['question']

In [76]:
import openai


BASE_URL = "https://philipp.openai.azure.com/"
API_KEY = ""
DEPLOYMENT_NAME = "gpt-4"

openai.api_key = API_KEY
openai.api_base = BASE_URL
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future



response = openai.ChatCompletion.create(
                engine="gpt-4",
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_message},
                ],
                temperature=0.2,
                max_tokens=500,
                n=1,
            )


cost = calc_cost(response)
completion = response.choices[0]['message']['content']

2023-11-03 16:14:53,065 INFO input_tokens=770, output_tokens=82, cost=0.07


In [64]:
code = extract_code(completion)

print(code)
exec(code, globals())



def max_problems(n, a):
    max_len = 1
    cur_len = 1
    for i in range(1, n):
        if a[i] <= 2 * a[i - 1]:
            cur_len += 1
        else:
            max_len = max(max_len, cur_len)
            cur_len = 1
    max_len = max(max_len, cur_len)
    return max_len



In [68]:

n = 6
a = [4, 7, 12, 100, 150, 199]

max_problems(n, a)

ans = [1]
t = 1

for i in range(n - 1):
    if a[i + 1] / a[i] <= 2:
        t += 1

    else:
        t = 1

    ans.append(t)

print(max(ans))

3
